In [ ]:
# | default_exp classes.DomoDataflow

In [ ]:
#| exporti
from __future__ import annotations

In [ ]:
# | exporti

from dataclasses import dataclass, field
import httpx

import domolibrary.utils.DictDot as util_dd

import domolibrary.client.DomoAuth as dmda
import domolibrary.routes.dataflow as dataflow_routes

from nbdev.showdoc import patch_to



In [ ]:
#| export
from domolibrary.classes.DomoDataflow_Action import DomoDataflow_Action
from domolibrary.classes.DomoDataflow_History import DomoDataflow_History, DomoDataflow_ActionResult

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
# | export
@dataclass
class DomoDataflow:
    id: str
    name: str = None
    auth: dmda.DomoAuth = field(default=None)
    owner: str = None
    description: str = None
    tags: list[str] = None
    actions: list[DomoDataflow_Action] = None
    
    history : DomoDataflow_History = None # class for managing the history of a dataflow

    def __post_init__(self):
        self.history = DomoDataflow_History(dataflow = self, dataflow_id = self. id, auth = self.auth)

In [ ]:
# | exporti
@patch_to(DomoDataflow, cls_method=True)
async def get_from_id(
    cls: DomoDataflow,
    dataflow_id: int,
    auth: dmda.DomoAuth = None,
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
):
    res = await dataflow_routes.get_dataflow_by_id(
        auth=auth,
        dataflow_id=dataflow_id,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=cls.__name__,
        session=session,
    )

    if return_raw:
        return res

    if not res.is_success:
        return None

    dd = util_dd.DictDot(res.response)

    domo_dataflow = cls(
        auth=auth,
        id=dd.id,
        name=dd.name,
        description=dd.description,
        owner=dd.owner,
        tags=dd.tags,
    )

    if dd.actions:
        domo_dataflow.actions = [
            DomoDataflow_Action._from_json(action) for action in dd.actions
        ]

    return domo_dataflow

#### sample implementation of get_from_id


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoDataflow.get_from_id(dataflow_id=70, auth=token_auth)

DomoDataflow(id=70, name='DomoStats - people - reports', auth=DomoTokenAuth(domo_instance='domo-community', token_name='token_auth', is_valid_token=True, url_manual_login='https://domo-community.domo.com/auth/index?domoManualLogin=true'), owner=None, description=None, tags=None, actions=[DomoDataflow_Action(id='241025d7-3cca-4369-b7c0-b3264277c0e1', type='LoadFromVault', depends_on=None, name='domostats_people', datasource_id='241025d7-3cca-4369-b7c0-b3264277c0e1', sql=None), DomoDataflow_Action(id='c176aa53-122a-4a2e-8b6f-41ca4a72d67c', type='GenerateTableAction', depends_on=['241025d7-3cca-4369-b7c0-b3264277c0e1'], name='t_1', datasource_id=None, sql='SELECT\n`Display Name`,\n`User ID`\n, @row := @row+1 as id\n, @isReset := case when @iter = @rowCounter then 1 else 0 end isReset\n, @group := @isReset + @group   as rowGroup\n, @iter := case when @isReset = 1 then 1 else @iter + 1 end iter\n, @rowCounter := case when @isReset = 1 then FLOOR( RAND() * (7-3) + 3) else @rowCounter end num

#### sample retrieval that also interacts with dataflow history from `DomoDataflow_History`
note will return actions history

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_dataflow = await DomoDataflow.get_from_id(dataflow_id=70, auth=token_auth)
await domo_dataflow.history.get_execution_history(maximum = 5)

[DomoDataflow_History_Execution(id=435276, dataflow_id=70, dataflow_execution_id='6ada1dc7-2629-4164-a74b-752567a03ca5', dataflow_version=262, begin_time=datetime.datetime(2023, 12, 7, 17, 14, 6), end_time=datetime.datetime(2023, 12, 7, 17, 14, 42), last_updated=datetime.datetime(2023, 12, 7, 17, 14, 42), is_failed=False, state='SUCCESS', activation_type='MANUAL', data_processor='MYSQL', telemetry=None, execution_stats={'total_bytes_written': 98048, 'total_rows_read': 910, 'total_bytes_read': 291559, 'mean_download_rate_kbps': 51.796541011006006, 'total_rows_written': 1818}, action_results=[DomoDataflow_ActionResult(id='8edc3d78-3586-47a8-9775-c7b146a50d19', type='PublishToVault', depends_on=None, name=None, datasource_id=None, sql=None, is_success=True, rows_processed=1818, begin_time=1701969276108, end_time=1701969279328), DomoDataflow_ActionResult(id='c176aa53-122a-4a2e-8b6f-41ca4a72d67c', type='GenerateTableAction', depends_on=None, name=None, datasource_id=None, sql=None, is_succe

In [ ]:
# | exporti
@patch_to(DomoDataflow)
async def execute(
    self: DomoDataflow,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
):
    return await dataflow_routes.execute_dataflow(
        auth=auth or self.auth,
        dataflow_id=self.id,
        parent_class=self.__class__.__name__,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

#### sample execute_dataflow


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_dataflow = DomoDataflow(id=70, auth=token_auth)

await domo_dataflow.execute()

ResponseGetData(status=200, response={'id': 435277, 'onboardFlowId': 70, 'previewRows': 0, 'dapDataFlowExecutionId': '629431b9-a742-432d-afb7-1863a1769e2a', 'beginTime': 1701969342374, 'lastUpdated': 1701969343966, 'state': 'CREATED', 'activationType': 'MANUAL', 'executionEngine': {'platform': 'K8S', 'engine': 'MYSQL_SQL_ENGINE'}, 'dataProcessor': 'MYSQL', 'dataFlowVersion': 262}, is_success=True, parent_class=None)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()